In [ ]:
import pandas as pd

dfproduct3 = pd.read_csv(
    r"C:\Users\pc\Desktop\AIbazaar\AIBazaar\AI\utils\newProduct.csv"
)
df = pd.read_csv(
    r"C:\Users\pc\Desktop\AIbazaar\AIBazaar\AI\utils\products_descriptions.csv"
)
df["ProductName"] = dfproduct3["Product Name"].str.strip()

df.to_csv("filterPhone2.csv", index=False)
print(f"Original rows: {len(df)}")

ValueError: Length of values (68) does not match length of index (370)